In [119]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
%matplotlib inline

# Import Data

In [120]:
training_set = pd.read_csv("Training Set.csv")
training_label = pd.read_csv("Training Set Labels.csv")
all_data = pd.merge(training_set, training_label, how="left", on="id")
print(all_data.shape)
all_data.head()

(59400, 41)


,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,3/14/11,Roman,1390,Roman,34.938093,-9.856322,none,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,3/6/13,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2/25/13,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,1/28/13,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,7/13/11,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


# Cleansing

In [121]:
# Drop columns
feature_list = ['amount_tsh', 'date_recorded', 'gps_height', 'longitude', 'latitude', 'basin', 'region', 'lga', 
                'population', 'extraction_type_group', 'extraction_type_class', 'payment_type', 'water_quality',
                'quantity', 'source_type', 'source_class', 'waterpoint_type', 'status_group']

all_data = all_data[feature_list]
print(all_data.shape)

(59400, 18)


In [122]:
# Transform date recorded column
date_recorded = pd.to_datetime(all_data["date_recorded"])
date_recorded.describe()

count                   59400
unique                    356
top       2011-03-15 00:00:00
freq                      572
first     2002-10-14 00:00:00
last      2013-12-03 00:00:00
Name: date_recorded, dtype: object

In [123]:
time_delta = (date_recorded - date_recorded.min()).astype('timedelta64[D]').astype(int) + 1
all_data["time_difference"] = time_delta
all_data = all_data.drop("date_recorded", axis=1)

In [124]:
# Remove outliers
clean_data = all_data[all_data["longitude"] != 0]

# Standardize continuous variables
continuous = ["amount_tsh", "gps_height", "longitude", "latitude", "population", "time_difference"]
scalar = preprocessing.StandardScaler()
scalar.fit(clean_data[continuous])
clean_data[continuous] = scalar.transform(clean_data[continuous])

print(clean_data.shape)

/Users/Fan/anaconda/envs/python35/lib/python3.5/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Fan/anaconda/envs/python35/lib/python3.5/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


(57588, 18)


In [125]:
# Quick check
clean_data["gps_height"].describe()

count    5.758800e+04
mean    -3.699008e-17
std      1.000009e+00
min     -1.123662e+00
25%     -9.938966e-01
50%     -3.796728e-01
75%      9.266344e-01
max      3.000001e+00
Name: gps_height, dtype: float64

In [126]:
# Generate dummy
categorical = ['basin', 'region', 'lga', 'extraction_type_group', 'extraction_type_class', 'payment_type',
               'water_quality', 'quantity', 'source_type', 'source_class', 'waterpoint_type']

for item in categorical:
    dummy = pd.get_dummies(clean_data[item], prefix=item)
    clean_data = clean_data.join(dummy.ix[:, :])
    clean_data = clean_data.drop(item, axis=1)

In [127]:
# Rearrange columns to make it look nicer
label = clean_data["status_group"]
clean_data = clean_data.drop("status_group", axis=1)
clean_data["status_group"] = label

# add column for Alexis
clean_data["is_functional"] = clean_data["status_group"].apply(lambda x: 1 if x == "functional" else 0)

# Quick check
print("Number of columns: ", clean_data.shape[1])
print("Number of rows: ", clean_data.shape[0])
clean_data.head(10)

Number of columns:  219
Number of rows:  57588


,amount_tsh,gps_height,longitude,latitude,population,time_difference,basin_Internal,basin_Lake Nyasa,basin_Lake Rukwa,basin_Lake Tanganyika,...,source_class_unknown,waterpoint_type_cattle trough,waterpoint_type_communal standpipe,waterpoint_type_communal standpipe multiple,waterpoint_type_dam,waterpoint_type_hand pump,waterpoint_type_improved spring,waterpoint_type_other,status_group,is_functional
0,1.863574,1.010261,-0.081144,-1.413152,-0.160277,-1.124130,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,functional,1
1,-0.107643,1.023238,-0.172932,1.330357,0.197658,1.027185,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,functional,1
2,-0.099430,-0.004794,0.886320,0.734645,0.134862,1.000405,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,functional,1
3,-0.107643,-0.614693,1.279621,-1.875446,-0.267030,0.917090,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,non functional,0
4,-0.107643,-0.993897,-1.541311,1.444992,-0.388435,-0.764090,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,functional,1
5,-0.101073,-0.993897,1.542962,0.398592,-0.386342,-1.127106,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,functional,1
6,-0.107643,-0.993897,-0.685455,0.754206,-0.388435,0.563001,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,non functional,0
7,-0.107643,-0.993897,-0.969950,0.590556,-0.388435,0.586805,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,non functional,0
8,-0.107643,-0.993897,-0.935247,0.262954,-0.388435,0.661193,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,non functional,0
9,-0.107643,-0.993897,-1.734551,1.647247,-0.388435,-0.701604,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,functional,1


In [128]:
# Write to csv
clean_data.to_csv("clean.csv")